In [289]:
# это тестовый символьный массив, чтобы можно было тестироваться
# pole_str = [['', '', ''],
#             ['', '', ''],
#             ['', '', '']]
pole_dig = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]  # объявляем цифровой массив, это пустое игровое поле

# копируем символьный массив в цифровой. был нужен для тестирования при разработке программы. сейчас не нужен
# for i in range(len(pole_str)):
#     for j in range(len(pole_str[i])):
#         if pole_str[i][j] == 'x':
#             pole_dig[i][j] = 1
#         elif pole_str[i][j] == 'o':
#             pole_dig[i][j] = 10

# функция печати поля из цифрового массива
def print_pole(polelocal):
    poleall = [' ', ' ', ' '], [' ', ' ', ' '], [' ', ' ', ' ']
    # копируем массив из цифр в массив символьный
    # i - это номер строки, j - это номер столбца
    for i in range(len(polelocal)):
        for j in range(len(polelocal[i])):
            if polelocal[i][j] == 1:
                poleall[i][j] = 'X'
            elif polelocal[i][j] == 10:
                poleall[i][j] = 'O'
    # печатаем игровое поле
    print('  0 1 2')
    for i in range(len(poleall)):
        print(i, poleall[i][0], poleall[i][1], poleall[i][2])

# функция ввода координат нолика пользователем. с обработкой ошибок ввода
def input_koord():
    while True:
        skoord = input('Введите через пробел строку и столбец, куда Вы хотите поставить 0:').split()
        xlocal = int(skoord[0]) if skoord[0] in ['0', '1', '2'] else 99
        if len(skoord) > 1:
            ylocal = int(skoord[1]) if skoord[1] in ['0', '1', '2'] else 99
            if xlocal != 99 and ylocal != 99:
                if pole_dig[xlocal][ylocal] == 0:
                    return xlocal, ylocal
                else:
                    print(pole_dig)
                    print('Вы не можете поставить 0 в эту клетку, т.к. она не пустая')
            else:
                print('Ошибка! Вы ввели', skoord[0], skoord[1], '. Необходимо вводить только цифры 0,1,2')
        else:
            print('Вы ввели только одно значение')

# проверка выигрыша на поле
def is_win(polelocal):
    win = 0  # 0 - ничья, 1-победа Х, 10-победа О, 99-есть пустые ячейки, игра не закончилась

    # проверям победные строки
    for i in range(len(polelocal)):
        # print(i,pole1[i][0],pole1[i][1],pole1[i][2])
        # print(pole[i])
        if sum(polelocal[i]) == 3:
            win = 1
        elif sum(polelocal[i]) == 30:
            win = 10

    # проверяем победные столбцы
    for j in range(len(polelocal[0])):
        if polelocal[0][j] + polelocal[1][j] + polelocal[2][j] == 3:
            win = 1
        elif polelocal[0][j] + polelocal[1][j] + polelocal[2][j] == 30:
            win = 10

    # проверяем победные диагонали
    if polelocal[0][0] + polelocal[1][1] + polelocal[2][2] == 3:
        win = 1
    elif polelocal[0][0] + polelocal[1][1] + polelocal[2][2] == 30:
        win = 10
    if polelocal[0][2] + polelocal[1][1] + polelocal[2][0] == 3:
        win = 1
    elif polelocal[0][2] + polelocal[1][1] + polelocal[2][0] == 30:
        win = 10

    # ищем пустые ячейки, если нашли, то игра еще не закончилась, иначе win остается равным нулю, т.е. ничья
    if not (win == 1 or win == 10):
        for i in range(len(polelocal)):
            for j in range(len(polelocal[i])):
                if polelocal[i][j] == 0:
                    win = 99

    return win

# записываем координаты пустых ячеек в массив
def coord_e_cell(coord_pole):
    coords = []
    for i in range(len(coord_pole)):
        for j in range(len(coord_pole[i])):
            if coord_pole[i][j] == 0:
                coords.append([i, j])
    return coords

# делаем ход человеком или компьютером, по заданной координате
def moving(pole1, man_or_comp, coord):
    pole_temp1 = pole1.copy()
    if man_or_comp == 'man':
        temp_hod = 10
    else:
        temp_hod = 1
    pole_temp1[coord[0]][coord[1]] = temp_hod
    return pole_temp1

# очищаем ячейку по заданной координате
def un_moving(pole1, coord):
    pole_temp1 = pole1.copy()
    pole_temp1[coord[0]][coord[1]] = 0
    return pole_temp1

# определяем врага относительно введенного значения. нужно для вызова рекурсивной функции,
# поочередно от компьютера и от человека
def kto_vrag(man_or_comp):
    if man_or_comp == 'man':
        return 'comp'
    else:
        return 'man'

# расчет лучшего хода по методу минимакса
def best_hod(pole2, man_or_comp):
    temp_win = is_win(pole2)
    if temp_win == 1:
        return 10, 0, 0
    elif temp_win == 10:
        return -10, 0, 0
    elif temp_win == 0:
        return 0, 0, 0

    best_hod_rating = 0
    coord_final=[]
    i=-1
    for temp_coord in coord_e_cell(pole2):
        pole2 = moving(pole2, man_or_comp, temp_coord)
        temp_rating, x, y = best_hod(pole2, kto_vrag(man_or_comp))
        pole2 = un_moving(pole2, temp_coord)
        if man_or_comp == 'comp':
            if temp_rating > best_hod_rating:
                best_hod_rating = temp_rating
        else:
            if temp_rating < best_hod_rating:
                best_hod_rating = temp_rating
        i+= 1
    coord_final=coord_e_cell(pole2)
    return best_hod_rating, coord_final[i][0], coord_final[i][1]

# общий метод выбора хода, чтобы проверить, не получится ли первый ход выигрышным или проигрышным
# если нет - то выбираем оптимальный ход
def metod(pole_metod):
    pole_temp=pole_metod.copy() #работаем с копией поля.
    coords_e_cell=coord_e_cell(pole_temp) #записываем в массив все пустые ячейки
    if len(coords_e_cell)==0:
        print('Ничья')

    #ходим Х-ом в любую из пустых ячеек, пока не получим выигрыш этим ходом
    for i in range(0,len(coords_e_cell)):
        pole_temp[coords_e_cell[i][0]][coords_e_cell[i][1]]=1
        if is_win(pole_temp)==1:
            pole_temp[coords_e_cell[i][0]][coords_e_cell[i][1]]=0
            print('Компьютер пошел',coords_e_cell[i][0],coords_e_cell[i][1],'. И победил!')
            return coords_e_cell[i][0],coords_e_cell[i][1]
        pole_temp[coords_e_cell[i][0]][coords_e_cell[i][1]]=0

    #ходим 0-ом в любую из пустых ячеек, пока не получим проигрыш. тогда ходим в эту ячейку
    for i in range(0,len(coords_e_cell)):
        pole_temp[coords_e_cell[i][0]][coords_e_cell[i][1]]=10
        if is_win(pole_temp)==10:
            pole_temp[coords_e_cell[i][0]][coords_e_cell[i][1]]=0
            print('Компьютер пошел',coords_e_cell[i][0],coords_e_cell[i][1])
            return coords_e_cell[i][0],coords_e_cell[i][1]
        pole_temp[coords_e_cell[i][0]][coords_e_cell[i][1]]=0

    #если предыдущими ходами не получили выигрыша или проигрыша - ищем лучший ход
    rat, x, y = best_hod(pole_dig, chey_hod)
    return x, y

# диалог ввода пользователем выбора того, кто начинает игру первым
def input_kto_perviy():
    while True:
        kto = input('Кто ходит первым? 1 - Человек, 2 - Компьютер:').split()
        kto_int = int(kto[0]) if kto[0] in ['1', '2'] else 99
        if len(kto) == 1:
            if kto_int == 1:
                return 'man'
            elif kto_int == 2:
                return 'comp'
            else:
                print('Ошибка! Вы ввели', kto[0], '. Необходимо вводить только цифры 1,2')
        else:
            print('Ошибка! Необходимо вводить только цифры 1,2')


chey_hod=input_kto_perviy()

print_pole(pole_dig)
if chey_hod == 'comp': # ходим в центральную ячейку, если компьютер ходит первым
    pole_dig[1][1] = 1
    chey_hod='man'
    print_pole(pole_dig)

while is_win(pole_dig) == 99:
    print('-------------------')
    if chey_hod == 'man':
        x, y = input_koord()
        pole_dig[x][y] = 10
        chey_hod = 'comp'
        if is_win(pole_dig) == 10:
            print('Вы победили!')

    else:
        if pole_dig[1][1] == 0:
            pole_dig[1][1] = 1 #ходим в центральную ячейку, если к этому моменту она еще пустая
        else:
            x,y=metod(pole_dig)
            pole_dig[x][y] = 1
        chey_hod = 'man'
    if is_win(pole_dig) == 0:
        print('Ничья')
    print_pole(pole_dig)




Кто ходит первым? 1 - Человек, 2 - Компьютер: 2


  0 1 2
0      
1      
2      
  0 1 2
0      
1   X  
2      
-------------------


Введите через пробел строку и столбец, куда Вы хотите поставить 0: 0 0


  0 1 2
0 O    
1   X  
2      
-------------------
  0 1 2
0 O    
1   X  
2     X
-------------------


Введите через пробел строку и столбец, куда Вы хотите поставить 0: 0 1


  0 1 2
0 O O  
1   X  
2     X
-------------------
Компьютер пошел 0 2
  0 1 2
0 O O X
1   X  
2     X
-------------------


Введите через пробел строку и столбец, куда Вы хотите поставить 0: 1 2


  0 1 2
0 O O X
1   X O
2     X
-------------------
Компьютер пошел 2 0 . И победил!
  0 1 2
0 O O X
1   X O
2 X   X
